# Libraries

In [20]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import cv2
import face_recognition

from PIL import Image

from sklearn.metrics import confusion_matrix
from mlxtend.plotting import plot_confusion_matrix

from keras import models
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D
from keras.optimizers import RMSprop,Adam
from keras.utils import to_categorical

In [21]:
data = pd.read_csv('./data/icml_face_data.csv')

# Functions

In [22]:
def prepare_data(data):
    """ Prepare data for modeling 
        input: data frame with labels and pixel data
        output: image and label array """
    
    image_array = np.zeros(shape=(len(data), 48, 48))
    image_label = np.array(list(map(int, data['emotion'])))
    
    for i, row in enumerate(data.index):
        image = np.fromstring(data.loc[row, ' pixels'], dtype=int, sep=' ')
        image = np.reshape(image, (48, 48))
        image_array[i] = image
        
    return image_array, image_label
        

# data prepare

In [23]:
emotions = {0: 'Angry', 1: 'Disgust', 2: 'Fear', 3: 'Happy', 4: 'Sad', 5: 'Surprise', 6: 'Neutral'}

Define training, validation and test data:

In [24]:
train_image_array, train_image_label = prepare_data(data[data[' Usage']=='Training'])
val_image_array, val_image_label = prepare_data(data[data[' Usage']=='PrivateTest'])
test_image_array, test_image_label = prepare_data(data[data[' Usage']=='PublicTest'])

Reshape and scale the images:

In [25]:
train_images = train_image_array.reshape((train_image_array.shape[0], 48, 48, 1))
train_images = train_images.astype('float32')/255
val_images = val_image_array.reshape((val_image_array.shape[0], 48, 48, 1))
val_images = val_images.astype('float32')/255
test_images = test_image_array.reshape((test_image_array.shape[0], 48, 48, 1))
test_images = test_images.astype('float32')/255

Encoding of the target value:

In [26]:
train_labels = to_categorical(train_image_label)
val_labels = to_categorical(val_image_label)
test_labels = to_categorical(test_image_label)

# Class Weights
Calculate the class weights of the label distribution:

In [27]:
class_weight = dict(zip(range(0, 7), (((data[data[' Usage']=='Training']['emotion'].value_counts()).sort_index())/len(data[data[' Usage']=='Training']['emotion'])).tolist()))

# Model

In [28]:
model = models.Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(48, 48, 1)))
model.add(MaxPool2D((2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPool2D((2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(Flatten())
model.add(Dense(64, activation='relu'))
model.add(Dense(7, activation='softmax'))

In [29]:
model.compile(optimizer=Adam(lr=1e-3), loss='categorical_crossentropy', metrics=['accuracy'])

In [30]:
history = model.fit(train_images, train_labels,
                    validation_data=(val_images, val_labels),
                    class_weight = class_weight,
                    epochs=12,
                    batch_size=64)

Epoch 1/12
449/449 [==============================] - 32s 71ms/step - loss: 0.2788 - accuracy: 0.2877 - val_loss: 1.5608 - val_accuracy: 0.3998
Epoch 2/12
449/449 [==============================] - 33s 73ms/step - loss: 0.2286 - accuracy: 0.4291 - val_loss: 1.4108 - val_accuracy: 0.4625
Epoch 3/12
449/449 [==============================] - 32s 70ms/step - loss: 0.2083 - accuracy: 0.4844 - val_loss: 1.3640 - val_accuracy: 0.4826
Epoch 4/12
449/449 [==============================] - 33s 73ms/step - loss: 0.1929 - accuracy: 0.5217 - val_loss: 1.2969 - val_accuracy: 0.5141
Epoch 5/12
449/449 [==============================] - 32s 71ms/step - loss: 0.1825 - accuracy: 0.5459 - val_loss: 1.2510 - val_accuracy: 0.5364
Epoch 6/12
449/449 [==============================] - 32s 71ms/step - loss: 0.1723 - accuracy: 0.5762 - val_loss: 1.2820 - val_accuracy: 0.5297
Epoch 7/12
449/449 [==============================] - 32s 72ms/step - loss: 0.1634 - accuracy: 0.5972 - val_loss: 1.2405 - val_accuracy:

In [31]:
test_loss, test_acc = model.evaluate(test_images, test_labels)

113/113 [==============================] - 1s 7ms/step - loss: 1.3599 - accuracy: 0.5447


In [32]:
pred_test_labels = model.predict(test_images)

In [33]:
pred_test_labels

array([[7.51804590e-01, 1.12143662e-02, 1.62660271e-01, ...,
        1.34748975e-02, 5.37384748e-02, 4.78524994e-03],
       [7.76912970e-03, 1.12994196e-04, 6.56801641e-01, ...,
        4.31676991e-02, 3.50601822e-02, 1.41103953e-01],
       [1.95718184e-01, 3.64321144e-03, 1.35279343e-01, ...,
        9.63828713e-02, 2.09218226e-02, 5.46407521e-01],
       ...,
       [4.28079278e-04, 9.47082981e-07, 3.36841494e-02, ...,
        3.19435708e-02, 2.09525004e-02, 3.98958437e-02],
       [5.81734776e-01, 3.86954239e-03, 3.41966264e-02, ...,
        8.75929072e-02, 5.29799238e-03, 2.56558567e-01],
       [9.85186081e-03, 4.59522380e-05, 4.16375436e-02, ...,
        4.44155373e-02, 5.77752106e-02, 8.43894362e-01]], dtype=float32)

# Save the Model

In [36]:
model.save('/Users/kimjiyoon/Driver-Protection-Service/AI/facial-emotion-detector/facial-emotion_model.h5')